In [1]:
import pandas as pd 
import numpy as np

In [34]:
dataset = pd.read_csv("D:\Datasets\pakwheels.csv")

In [35]:
# this is dropping the columns form the dataframe
final_data_frame = dataset.drop(['post_date','price_category','mileage_category','post_day_of_week'],axis=1)

In [36]:
# calculating the mean of price feature form the dataframe
average_price = final_data_frame.loc[final_data_frame['price'] != 0 ,'price'].mean()

In [37]:
# replace the mean that we calculated earlier with the zeros present in the price column in dataframe
final_data_frame['price'] = final_data_frame['price'].replace(0 , average_price)

In [38]:
# we were encountering a problem in which there some e+ in price column data
final_data_frame['price'] = final_data_frame['price'].apply(lambda x: f'{x:.2f}')

In [39]:
# To select the first three words of the title form the title column 
final_data_frame['title'] = final_data_frame['title'].str.split(' ').str.slice(0,2).str.join(' ')

In [40]:
# importing some important liabraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [41]:
#basically we are seeing the vast categories of title category in the title column of dataframe
# here we are seeing how much there is rows about a certain car name and storing it into max variable
max = final_data_frame['title'].value_counts()

In [42]:
# here we are taking 24 most frequent categories of title column and storing it into the variable filtered
filtered = max.head(24) 


In [43]:
# this line of code is putting 
filtered_df = final_data_frame[final_data_frame['title'].isin(filtered.index)]

In [44]:
# here we are getting the x and y for xy splitting
X = filtered_df.drop(['price'], axis=1)
Y = filtered_df['price']

In [45]:
# resetting the index the columns
y = Y.reset_index(drop=True)
x = X.reset_index(drop=True)

In [46]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y , test_size=0.2, random_state=42)


In [47]:
ColumnT= ColumnTransformer(transformers=[("name",OneHotEncoder(sparse_output=False,handle_unknown='ignore'),['title','city','fuel_type','transmission','registered','color','assembly'])])

In [48]:
X_transformed_train = ColumnT.fit_transform(X_train)
X_transformed_test = ColumnT.transform(X_test)

In [49]:
model = LinearRegression()

In [50]:
pipe = Pipeline(steps=[
    ('preprocessor', ColumnT),
    ('model', LinearRegression())
])

In [51]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('name',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['title', 'city', 'fuel_type',
                                                   'transmission', 'registered',
                                                   'color', 'assembly'])])),
                ('model', LinearRegression())])

In [52]:
Y_pred = pipe.predict(X_test)
Y_train_pred = pipe.predict(X_train)

In [53]:
def predict_price(title, city, model,mileage,fuel_type, transmission, registered, color, assembly,engine_capacity,vehicle_age ):
    # Create a DataFrame with the input data
    input_data = pd.DataFrame({
        'title': [title],
        'city': [city],
        'model':[model],
        'mileage':[mileage],
        'fuel_type': [fuel_type],
        'transmission': [transmission],
        'registered': [registered],
        'color': [color],
        'assembly': [assembly],
        'engine_capacity':[engine_capacity],
        'vehicle_age':[vehicle_age],
    })
    
    # Use the pipeline to predict the price
    predicted_price = pipe.predict(input_data)
    
    # Return the predicted price
    return predicted_price[0]

In [54]:
 predicted_price = predict_price(
    title='Suzuki Mehran', 
    city='Multan', 
    model= 2016,
    mileage=100000,
    fuel_type='Petrol', 
    transmission='Not Available', 
    registered='Multan', 
    color='White', 
    assembly='Local',
    engine_capacity=1000,
    vehicle_age=8
)
    

In [55]:
predicted_price

1216512.0

In [56]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [57]:
mse = mean_squared_error(Y_train, Y_train_pred)
mae = mean_absolute_error(Y_train, Y_train_pred)
r2 = r2_score(Y_train, Y_train_pred)

In [27]:
r2

0.6615970317460534

In [28]:
mae

1429515.2316968492

In [29]:
filtered_df['title'].value_counts()

title
Toyota Corolla     7306
Honda Civic        4637
Honda City         3360
Suzuki Mehran      2684
Suzuki Alto        2489
Suzuki Cultus      2298
Suzuki Wagon       1326
Toyota Vitz        1237
Suzuki Swift        941
Toyota Yaris        940
Honda Vezel         802
Daihatsu Mira       754
KIA Sportage        746
Toyota Passo        700
Toyota Prado        688
Toyota Prius        660
Suzuki Bolan        655
Toyota Hilux        617
Toyota Land         611
Toyota Aqua         580
Mercedes Benz       495
Toyota Fortuner     488
Daihatsu Cuore      412
Suzuki Every        398
Name: count, dtype: int64